# 3. Evolve-Filter, steady state

* $\delta = 0.1, 0.01, 0.00125, 1/nx$
* $a = a_D$

In [1]:
%matplotlib inline 
import matplotlib
from dolfin import *
from __future__ import print_function
import numpy as np
import sympy as sym
import csv

x, y, t = sym.symbols('x[0], x[1], t')
sigma = 1.0
mu = 10**(-5) 
b=as_vector([2.0, 3.0])
# Exact Solution
c = 16.0 #*sym.sin(sym.pi*t)
h = x*(1-x)*y*(1-y)
g = 2*mu**(-0.5)*(0.25**2 - (x - 0.5)**2 - (y - 0.5)**2 )
iliescu = c*h*(0.5+sym.atan(g)/sym.pi)

ue = sym.simplify(iliescu)
u_code = sym.printing.ccode(ue)
u_code = u_code.replace('M_PI','DOLFIN_PI')

# du/dt - mu*Laplace(u) + div(bu) + sigma*u = f
fe = 0 #sym.diff(ue,t) Remove time dependence for now
fe += - mu*(sym.diff(sym.diff(ue,x),x) + sym.diff(sym.diff(ue,y),y))
fe += b[0]*sym.diff(ue,x) + b[1]*sym.diff(ue,y)
fe += sigma*ue

f_code = sym.printing.ccode(fe)
f_code = f_code.replace('M_PI','DOLFIN_PI')

In [2]:
# %load main-EFR.py

def compute_errors(u_e, u, t, mesh):
	L2n = errornorm(u_e, u, norm_type='L2', degree_rise=3, mesh=mesh)
	H1n = errornorm(u_e, u, norm_type='H1', degree_rise=3, mesh=mesh)
	return L2n, H1n

def compute_extrema(u, t):
    maxval = np.amax(u.vector().get_local())
    minval = np.amin(u.vector().get_local())
    return maxval, minval

In [3]:
def evolve_filter(filename, delta, nx, u_code, f_code, sigma, mu, velocity):
    degree = 2
    folder = 'EF_aD/'

    
    u_exact = Expression(u_code, degree = degree)  
    f = Expression(f_code, degree = degree)
    mesh = UnitSquareMesh(nx,nx)
    Q = FunctionSpace(mesh, "CG", degree)

    # Set up boundary condition
    u_D = Expression(u_exact.cppcode, degree = degree)
    def boundary(x, on_boundary):
        return on_boundary
    
# ------------ STEP 1: Solve on coarse grid ----------------------#

    bc = DirichletBC(Q, u_D, boundary)

    # Test and trial functions
    u, v = TrialFunction(Q), TestFunction(Q)
    u_n = Function(Q)
    u_ = Function(Q)

    # Galerkin variational problem
    F = mu*dot(grad(v), grad(u))*dx + v*dot(velocity, grad(u))*dx + sigma*v*u*dx - f*v*dx

    # Create bilinear and linear forms
    a1 = lhs(F)
    L = rhs(F)

    # Assemble matrices
    A1 = assemble(a1)
    bc.apply(A1)

    # Create progress bar
    progress = Progress('Time-stepping')
    set_log_level(PROGRESS)

    # Step 1 Solve on Coarse Grid
    b = assemble(L)
    bc.apply(b)
    solve(A1, u_.vector(), b, 'gmres')
    
#     errors = compute_errors(u_exact, u_, 0, mesh)
#     print(errors)

    out_file_uexact = File(folder+"evolve_steady_u_exact_"+str(nx)+".pvd") 
    ue = interpolate(u_exact, Q)
    out_file_uexact << ue
    out_file_ubar = File(folder+str(mu)+"evolve_steady_u_"+str(nx)+".pvd") 
    out_file_ubar << u_

    
# ------------ STEP 2: Helmholtz filter ----------------------#
    
    # Helmholtz filter to compute the indicator function
    u_tilde = TrialFunction(Q)

    F_Hfilter = v*u_tilde*dx + delta*delta*dot(grad(v), grad(u_tilde))*dx - v*u_*dx 

    a_Hfilter = lhs(F_Hfilter)
    L_Hfilter = rhs(F_Hfilter)

    A_Hfilter = assemble(a_Hfilter)
    bc.apply(A_Hfilter)

    b_Hfilter = assemble(L_Hfilter)
    bc.apply(b_Hfilter)

    solver = LUSolver(A_Hfilter)
    u_tilde = Function(Q)
    solver.solve(u_tilde.vector(), b_Hfilter)

#     out_file_utilde << u_tilde

    # Compute the indicator function N = 0
    indicator = Expression('sqrt((a-b)*(a-b))', degree = 2, a = u_, b = u_tilde)
    indicator = interpolate(indicator, Q)
    max_ind = np.amax(indicator.vector().get_local())

    if max_ind < 1:
        max_ind = 1.0

    indicator = Expression('a/b', degree = 2, a = indicator, b = max_ind)
    indicator = interpolate(indicator, Q)

    out_file_ind = File(folder+str(mu)+"EF_aD"+str(nx)+".pvd")
    out_file_ind << indicator

    # Apply the filter
    u_bar = TrialFunction(Q)
    F_filter = v*u_bar*dx + delta*delta*dot(grad(v), indicator*grad(u_bar))*dx - v*u_*dx 

    a_filter = lhs(F_filter)
    L_filter = rhs(F_filter)

    A_filter = assemble(a_filter)
    bc.apply(A_filter)

    b_filter = assemble(L_filter)
    bc.apply(b_filter)

    solver = LUSolver(A_filter)
    u_bar = Function(Q)
    solver.solve(u_bar.vector(), b_filter)
    
    out_file_ufilter = File(folder+"delta"+filename+"_filter_u_"+str(nx)+".pvd") 
    out_file_ufilter << u_bar

    
    L2, H1 = compute_errors(u_exact, u_bar, t, mesh)
    maxval, minval = compute_extrema(u_bar, t)
    
    print(nx,",",L2,",",H1,",",maxval,",",minval)


## $ N = 0 $

L2 = [0.281849030719,0.174892605469,0.029634489156,0.0051976175589,0.000418226657031,0];

H1 = [15.6615596083,12.864296479,7.49899879463,3.32468881229,0.643213010063,0];


In [4]:
filename = 'varies'
for nx in [25, 50, 100, 200, 400]:
    delta = 1.0/nx
    evolve_filter(filename, delta, nx, u_code, f_code, sigma, mu, b)

25 , 0.281849030719 , 15.6615596083 , 1.58232307758 , -0.77453060737
50 , 0.174892605469 , 12.864296479 , 1.48156398067 , -0.518960941244
100 , 0.029634489156 , 7.49899879463 , 1.07818906133 , -0.104262476451
200 , 0.0051976175589 , 3.32468881229 , 1.00559701449 , -0.0300435478052
400 , 0.000418226657031 , 0.643213010063 , 0.992008662377 , -0.000933532363759
